In [349]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
from scipy.special.cython_special import nbdtr

In [350]:
players = nfl.import_seasonal_rosters([2024])
print(players.columns)
picks = nfl.import_draft_values()
print(picks)

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'player_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'ngs_position', 'week',
       'game_type', 'status_description_abbr', 'football_name', 'esb_id',
       'gsis_it_id', 'smart_id', 'entry_year', 'rookie_year', 'draft_club',
       'draft_number', 'age'],
      dtype='object')
     pick  stuart  johnson     hill   otc    pff
0       1    34.6     3000  1000.00  3000  1.135
1       2    30.2     2600   717.17  2635  1.099
2       3    27.6     2200   514.33  2421  1.063
3       4    25.8     1800   490.52  2270  1.027
4       5    24.3     1700   467.81  2152  0.991
..    ...     ...      ...      ...   ...    ...
257   258     0.0        0     1.08    76  0.047
258   259     0.0        0  

In [351]:
players.rename(columns={'draft_number': 'pick'}, inplace=True)
players['pick'] = players['pick'].fillna(0).astype(int)
print(players)

      season team position depth_chart_position  jersey_number status  \
0       2024  SEA       OL                    T           70.0    DEV   
1       2024  NYJ       QB                   QB            8.0    ACT   
2       2024  SEA        P                    P            9.0    RET   
3       2024  ARI        K                    K            5.0    RES   
4       2024  CHI       TE                   TE           84.0    ACT   
...      ...  ...      ...                  ...            ...    ...   
3210    2024  IND       LB                   LB           36.0    CUT   
3211    2024  SEA       TE                   TE           46.0    CUT   
3212    2024   NE       DL                   DE           30.0    CUT   
3213    2024  CAR       LB                   LB           41.0    CUT   
3214    2024   NO       DL                   DT           76.0    RES   

              player_name first_name  last_name birth_date  ...  \
0            Jason Peters      Jason     Peters 1982-01-

In [352]:
player_draft_data = pd.merge(picks, players, on= 'pick', how='right')
print(player_draft_data)
print(player_draft_data.columns)

      pick  stuart  johnson    hill     otc    pff  season team position  \
0        0     NaN      NaN     NaN     NaN    NaN    2024  SEA       OL   
1       24    14.4    740.0  237.25  1326.0  0.565    2024  NYJ       QB   
2        0     NaN      NaN     NaN     NaN    NaN    2024  SEA        P   
3        0     NaN      NaN     NaN     NaN    NaN    2024  ARI        K   
4       28    13.4    660.0  209.11  1245.0  0.533    2024  CHI       TE   
...    ...     ...      ...     ...     ...    ...     ...  ...      ...   
3210     0     NaN      NaN     NaN     NaN    NaN    2024  IND       LB   
3211     0     NaN      NaN     NaN     NaN    NaN    2024  SEA       TE   
3212     0     NaN      NaN     NaN     NaN    NaN    2024   NE       DL   
3213     0     NaN      NaN     NaN     NaN    NaN    2024  CAR       LB   
3214     0     NaN      NaN     NaN     NaN    NaN    2024   NO       DL   

     depth_chart_position  ...  game_type status_description_abbr  \
0                 

In [353]:
player_draft_data = player_draft_data.loc[:,['season', 'team',
       'position', 'player_name', 'player_id', 'years_exp', 'entry_year', 'rookie_year', 'draft_club',
       'age', 'pick','pff']]

print(player_draft_data[player_draft_data['player_name'] == 'Aaron Donald'])
print(player_draft_data)

     season team position   player_name   player_id  years_exp  entry_year  \
119    2024   LA       DL  Aaron Donald  00-0031388         10        2014   

     rookie_year draft_club   age  pick    pff  
119       2014.0        STL  33.0    13  0.715  
      season team position          player_name   player_id  years_exp  \
0       2024  SEA       OL         Jason Peters  00-0022531         20   
1       2024  NYJ       QB        Aaron Rodgers  00-0023459         19   
2       2024  SEA        P             Jon Ryan  00-0023742         18   
3       2024  ARI        K          Matt Prater  00-0023853         18   
4       2024  CHI       TE       Marcedes Lewis  00-0024243         18   
...      ...  ...      ...                  ...         ...        ...   
3210    2024  IND       LB       Mike Smith Jr.  00-0039951          0   
3211    2024  SEA       TE       Devon Garrison  00-0039952          0   
3212    2024   NE       DL  Christian McCarroll  00-0039953          0   
3213 

In [354]:
print(player_draft_data['position'].unique())

['OL' 'QB' 'P' 'K' 'TE' 'LS' 'DL' 'DB' 'LB' 'RB' 'WR']


In [355]:
LB_positions = ['ILB', 'OLB', 'MLB', 'LB']
CB_positions = ['CB', 'FS', 'SS', 'DB']
DL_positions = ['DE', 'DT', 'NT']
OL_positions = ['C', 'G', 'T']
RB_positions = ['RB', 'FB']
DROP_positions = ['LS', 'PR', 'P', 'K']

player_draft_data['position'] = player_draft_data['position'].replace(LB_positions, 'LB')
player_draft_data['position'] = player_draft_data['position'].replace(CB_positions, 'CB')
player_draft_data['position'] = player_draft_data['position'].replace(DL_positions, 'DL')
player_draft_data['position'] = player_draft_data['position'].replace(OL_positions, 'OL')
player_draft_data['position'] = player_draft_data['position'].replace(RB_positions, 'RB')

player_draft_data = player_draft_data[~player_draft_data['position'].isin(DROP_positions)]

In [356]:
print(player_draft_data['team'].unique())

['SEA' 'NYJ' 'CHI' 'ATL' 'IND' 'MIA' 'BAL' 'LA' 'SF' 'PHI' 'BUF' 'HOU'
 'DAL' 'NO' 'PIT' 'CAR' 'CIN' 'WAS' 'CLE' 'ARI' 'MIN' 'TB' 'DET' 'NYG'
 'LV' 'LAC' 'KC' 'JAX' 'DEN' 'TEN' 'NE' 'GB']


In [357]:
team_mapping = {
    'ARI': 'Cardinals',
    'ARZ': 'Cardinals',
    'ATL': 'Falcons',
    'BLT': 'Ravens',
    'BAL': 'Ravens',
    'BUF': 'Bills',
    'CAR': 'Panthers',
    'CHI': 'Bears',
    'CIN': 'Bengals',
    'CLV': 'Browns',
    'CLE': 'Browns',
    'DAL': 'Cowboys',
    'DEN': 'Broncos',
    'DET': 'Lions',
    'GB': 'Packers',
    'HST': 'Texans',
    'HOU': 'Texans',
    'IND': 'Colts',
    'JAX': 'Jaguars',
    'KC': 'Chiefs',
    'LAC': 'Chargers',
    'SD': 'Chargers',
    'LA': 'Rams',
    'SL': 'Rams',
    'MIA': 'Dolphins',
    'MIN': 'Vikings',
    'NE': 'Patriots',
    'NO': 'Saints',
    'NYG': 'Giants',
    'NYJ': 'Jets',
    'PHI': 'Eagles',
    'PIT': 'Steelers',
    'SF': '49ers',
    'SEA': 'Seahawks',
    'TB': 'Buccaneers',
    'TEN': 'Titans',
    'WAS': 'Commanders',
    'OAK': 'Raiders',
    'LV': 'Raiders'
}
player_draft_data['team'] = player_draft_data['team'].replace(team_mapping)


In [358]:
player_draft_data['pff'] = player_draft_data['pff'].fillna(0).astype(int)
k_values = (1 / 4)
player_draft_data['adjusted_value'] = player_draft_data['pff'] * np.exp(-k_values * player_draft_data['years_exp'])

In [359]:
player_draft_data = player_draft_data.sort_values(by=['team', 'position', 'season'], ascending=False)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(player_draft_data)

      season        team position                 player_name   player_id  \
587     2024     Vikings       WR             Trent Sherfield  00-0034487   
610     2024     Vikings       WR              Brandon Powell  00-0034646   
996     2024     Vikings       WR            Trishton Jackson  00-0035934   
1155    2024     Vikings       WR            Justin Jefferson  00-0036322   
1688    2024     Vikings       WR                Jalen Nailor  00-0037291   
1793    2024     Vikings       WR                 Justin Hall  00-0037508   
1995    2024     Vikings       WR               Lucky Jackson  00-0038181   
2304    2024     Vikings       WR               Thayer Thomas  00-0038810   
2404    2024     Vikings       WR              Jordan Addison  00-0038994   
2976    2024     Vikings       WR               Devron Harper  00-0039708   
2977    2024     Vikings       WR                    Ty James  00-0039709   
2978    2024     Vikings       WR               Jeshaun Jones  00-0039710   